In [1]:
!pip install timm

     |████████████████████████████████| 225 kB 404 kB/s 


In [2]:
from typing import Dict

from tempfile import gettempdir
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
#from torchvision.models.resnet import resnet50, resnet18, resnet34, resnet101
from tqdm import tqdm

import l5kit
from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

import matplotlib.pyplot as plt

import os
import random
import time
import timm
import warnings
warnings.filterwarnings("ignore")

In [3]:
l5kit.__version__

'1.1.0'

In [4]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    
set_seed(42)

In [5]:
cfg = {
    'format_version': 4,
    'data_path': "../input/lyft-motion-prediction-autonomous-vehicles",
    'model_params': {
        'model_architecture': 'resnest101',
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1,
        'model_name': "lyft_res2net",
        'lr': 1e-3,
        'weight_path': "../input/res2net-lyft/lyft_res2net_1000.pth",
        'train': True,
        'predict': False
    },

    'raster_params': {
        'raster_size': [225, 225],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },

    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 96,
        'shuffle': True,
        'num_workers': 4
    }, 
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4
    },

    'train_params': {
        'max_num_steps': 10000,
        'checkpoint_every_n_steps': 500,
        'image_coords':True
    },
    'test_params':{'image_coords':True}
}

In [6]:
# set env variable for data
DIR_INPUT = cfg["data_path"]
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)

In [7]:
# ===== INIT TRAIN DATASET============================================================
if cfg['model_params']['train']:
  train_cfg = cfg["train_data_loader"]
  rasterizer = build_rasterizer(cfg, dm)
  train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
  train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
  train_dataloader = DataLoader(train_dataset, shuffle=train_cfg["shuffle"], batch_size=train_cfg["batch_size"], 
                              num_workers=train_cfg["num_workers"])
  print("==================================TRAIN DATA==================================")
  print(train_dataset)

==================================TRAIN DATA==================================
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Num TR lights | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |    38735988   |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+---------------+-----------------+----------------------+----------------------+----------------------+---------------------+


In [8]:
cfg['train_params']['max_num_steps'] = len(train_dataloader)

In [9]:
if cfg['model_params']['predict']:
#====== INIT TEST DATASET=============================================================
    test_cfg = cfg["test_data_loader"]
    rasterizer = build_rasterizer(cfg, dm)
    test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
    test_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
    test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
    test_dataloader = DataLoader(test_dataset,shuffle=test_cfg["shuffle"],batch_size=test_cfg["batch_size"],
                                 num_workers=test_cfg["num_workers"])
    print("==================================TEST DATA==================================")
    print(test_dataset)

In [10]:
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
import numpy as np

import torch
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

In [11]:
class LyftMultiModel(nn.Module):

    def __init__(self,model_name, cfg: Dict, num_modes=3):
        super().__init__()

        self.backbone = timm.create_model(model_name)

        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )

        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets
        
        backbone_out_features = 2048

        # X, Y coords for the future positions (output shape: batch_sizex50x2)
        self.future_len = cfg["model_params"]["future_num_frames"]
        num_targets = 2 * self.future_len

        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )

        self.num_preds = num_targets * num_modes
        self.num_modes = num_modes

        self.logit = nn.Linear(4096, out_features=self.num_preds + num_modes)

    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.act1(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.global_pool(x)
        x = torch.flatten(x, 1)

        x = self.head(x)
        x = self.logit(x)

        # pred (batch_size)x(modes)x(time)x(2D coords)
        # confidences (batch_size)x(modes)
        bs, _ = x.shape
        pred, confidences = torch.split(x, self.num_preds, dim=1)
        pred = pred.view(bs, self.num_modes, self.future_len, 2)
        assert confidences.shape == (bs, self.num_modes)
        confidences = torch.softmax(confidences, dim=1)
        return pred, confidences

In [12]:
def forward(data, model, device, criterion = pytorch_neg_multi_log_likelihood_batch):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)
    # Forward pass
    preds, confidences = model(inputs)
    loss = criterion(targets, preds, confidences, target_availabilities)
    return loss, preds, confidences

In [13]:
'''def forward(data, model, device, criterion = pytorch_neg_multi_log_likelihood_batch):
    inputs = data["image"].to(device)
    target_availabilities = data["target_availabilities"].to(device)
    targets = data["target_positions"].to(device)
    matrix = data["world_to_image"].to(device)
    centroid = data["centroid"].to(device)[:,None,:].to(torch.float)

    # Forward pass
    outputs = model(inputs)

    bs,tl,_ = targets.shape
    assert tl == cfg["model_params"]["future_num_frames"]

    if cfg['train_params']['image_coords']:
        targets = targets + centroid
        targets = torch.cat([targets,torch.ones((bs,tl,1)).to(device)], dim=2)
        targets = torch.matmul(matrix.to(torch.float), targets.transpose(1,2))
        targets = targets.transpose(1,2)[:,:,:2]
        bias = torch.tensor([56.25, 112.5])[None,None,:].to(device)
        targets = targets - bias
#     print(outputs)
    confidences, pred = outputs[1], outputs[0]
    pred = pred.view(bs, 3, tl, 2)
    assert confidences.shape == (bs, 3)
    confidences = torch.softmax(confidences, dim=1)

    loss = criterion(targets, pred, confidences, target_availabilities)
    loss = torch.mean(loss)

    if cfg['train_params']['image_coords']:
        matrix_inv = torch.inverse(matrix)
        pred = pred + bias[:,None,:,:]
        pred = torch.cat([pred,torch.ones((bs,3,tl,1)).to(device)], dim=3)
        pred = torch.stack([torch.matmul(matrix_inv.to(torch.float), pred[:,i].transpose(1,2)) 
                            for i in range(3)], dim=1)
        pred = pred.transpose(2,3)[:,:,:,:2]
        pred = pred - centroid[:,None,:,:]

    return loss, pred, confidences'''

'def forward(data, model, device, criterion = pytorch_neg_multi_log_likelihood_batch):\n    inputs = data["image"].to(device)\n    target_availabilities = data["target_availabilities"].to(device)\n    targets = data["target_positions"].to(device)\n    matrix = data["world_to_image"].to(device)\n    centroid = data["centroid"].to(device)[:,None,:].to(torch.float)\n\n    # Forward pass\n    outputs = model(inputs)\n\n    bs,tl,_ = targets.shape\n    assert tl == cfg["model_params"]["future_num_frames"]\n\n    if cfg[\'train_params\'][\'image_coords\']:\n        targets = targets + centroid\n        targets = torch.cat([targets,torch.ones((bs,tl,1)).to(device)], dim=2)\n        targets = torch.matmul(matrix.to(torch.float), targets.transpose(1,2))\n        targets = targets.transpose(1,2)[:,:,:2]\n        bias = torch.tensor([56.25, 112.5])[None,None,:].to(device)\n        targets = targets - bias\n#     print(outputs)\n    confidences, pred = outputs[1], outputs[0]\n    pred = pred.view(

In [14]:
!pip install adamp
from adamp import AdamP

  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5998 sha256=08878b6b4a19b7d06d4222bdf7b1cc6c8889b63648dad71696ed81054d9ab6dd
  Stored in directory: /root/.cache/pip/wheels/bb/95/21/ced2d2cb9944e3a72e58fece7958973eed3fd8d0aeb6e2e450
Successfully built adamp


In [15]:
# ==== INIT MODEL=================
import seaborn as sns
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LyftMultiModel('res2net50_14w_8s',cfg)
# model1 = LyftMultiModel('res2net50_26w_4s',cfg)

#load weight if there is a pretrained model
weight_path = cfg["model_params"]["weight_path"]

model.to(device)
# model1.to(device)

optimizer = AdamP(model.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-2)
# optimizer1 = AdamP(model1.parameters(), lr=0.001, betas=(0.9, 0.999), weight_decay=1e-2)
print(f'device {device}')
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, len(train_dataloader), eta_min=0.1)
# scheduler1 = optim.lr_scheduler.CosineAnnealingLR(optimizer1, len(train_dataloader), eta_min=0.1)

device cuda:0


In [16]:

if weight_path:
    ckpt = torch.load(weight_path)
    optimizer.load_state_dict(ckpt['res2net_16w_8s']['optimizer_state_dict'])
    scheduler.load_state_dict(ckpt['res2net_16w_8s']['scheduler_state_dict'])
#     optimizer1.load_state_dict(ckpt['res2net_26w_4s']['optimizer_state_dict'])
#     scheduler1.load_state_dict(ckpt['res2net_26w_4s']['scheduler_state_dict'])
    model.load_state_dict(ckpt['res2net_16w_8s']['model_state_dict'])
#     model1.load_state_dict(ckpt['res2net_26w_4s']['model_state_dict'])
    iterno = ckpt['iter']
    del ckpt


In [17]:
if cfg["model_params"]["train"]:  
    tr_it = iter(train_dataloader)
    bf = 0
    num_iter = cfg["train_params"]["max_num_steps"]
    losses_train = []
    losses_train1 = []
    iterations = []
    metrics = []
    times = []
    model_name = cfg["model_params"]["model_name"]
    start = time.time()

    progress_bar = tqdm(range(iterno,cfg["train_params"]["max_num_steps"]))
    for i in progress_bar:
        try:
            data = next(tr_it)
        except StopIteration:
            tr_it = iter(train_dataloader)
            data = next(tr_it)
        model.train()
#         model1.train()
        torch.set_grad_enabled(True)

        loss, _, _ = forward(data, model, device)
#         loss1, _, _ = forward(data, model1, device)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
#         optimizer1.zero_grad()
#         loss1.backward()
#         optimizer1.step()
#         scheduler1.step()

        losses_train.append(loss.item())
#         losses_train1.append(loss1.item())

        progress_bar.set_description(f"loss: {loss.item()} loss(avg): {np.mean(losses_train)}")
#       :    loss: {loss1.item()} loss(avg): {np.mean(losses_train1)}
        if i % cfg['train_params']['checkpoint_every_n_steps'] == 0:
            torch.save({
                        'res2net_16w_8s':{'model_state_dict':model.state_dict(),
                                          'optimizer_state_dict':optimizer.state_dict(),
                                          'scheduler_state_dict':scheduler.state_dict()},
#                         'res2net_26w_4s':{'model_state_dict':model1.state_dict(),
#                                           'scheduler_state_dict':scheduler1.state_dict(),
#                                           'optimizer_state_dict':optimizer1.state_dict()},
                        'iter':i+1}, f'./lyft_res2net_{i}.pth')
            os.system(f'rm ./lyft_res2net_{bf}.pth')
            bf = i
            iterations.append(i)
            metrics.append(np.mean(losses_train))
            times.append((time.time()-start)/60)
        if i % 5000 == 0 : 
            results = pd.DataFrame({'iterations': iterations, 'metrics (avg)': metrics, 'elapsed_time (mins)': times})
            results.to_csv(f"train_metrics_{model_name}_{num_iter}.csv", index = False)
            print(f"Total training time is {(time.time()-start)/60} mins")
            print(results.head())
            data = pd.DataFrame({'iter':list(range(len(losses_train))),'losses':losses_train})
            sns.scatterplot(x = 'iter',y = 'losses',data = data,hue = 'losses')
            plt.show()

loss: 3410.59716796875 loss(avg): 3410.59716796875:   0%|          | 1/233340 [00:49<3228:47:31, 49.81s/it]


AssertionError: confidences should sum to 1

In [18]:
# ==== EVAL LOOP ================================================================
if cfg["model_params"]["predict"]:
    
    model.eval()
    torch.set_grad_enabled(False)

    # store information for evaluation
    future_coords_offsets_pd = []
    timestamps = []
    confidences_list = []
    agent_ids = []

    progress_bar = tqdm(test_dataloader)
    
    for data in progress_bar:
        inputs = data["image"].to(device)
        #target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
        target_availabilities = data["target_availabilities"].to(device)
        targets = data["target_positions"].to(device)
        matrix = data["world_to_image"].to(device)
        centroid = data["centroid"].to(device)[:,None,:].to(torch.float)

        bs,tl,_ = targets.shape
        rs = cfg["raster_params"]["raster_size"]
        ec = cfg["raster_params"]["ego_center"]
        bias = torch.tensor([56.25, 112.5])[None, None, :].to(device)


        #outputs = model(inputs).reshape(targets.shape)
        pred, confidences = model(inputs)

        if cfg['test_params']['image_coords']:
            matrix_inv = torch.inverse(matrix)
            pred = pred + bias[:,None,:,:]
            pred = torch.cat([pred,torch.ones((bs,3,tl,1)).to(device)], dim=3)
            pred = torch.stack([torch.matmul(matrix_inv.to(torch.float), pred[:,i].transpose(1,2)) 
                                for i in range(3)], dim=1)
            pred = pred.transpose(2,3)[:,:,:,:2]
            pred = pred - centroid[:,None,:,:]
    
        future_coords_offsets_pd.append(pred.cpu().numpy())
        confidences_list.append(confidences.cpu().numpy())
        timestamps.append(data["timestamp"].numpy())
        agent_ids.append(data["track_id"].numpy()) 

In [19]:
#create submission to submit to Kaggle
pred_path = 'submission.csv'
write_pred_csv(pred_path,
           timestamps=np.concatenate(timestamps),
           track_ids=np.concatenate(agent_ids),
           coords=np.concatenate(future_coords_offsets_pd),
           confs = np.concatenate(confidences_list)
          )

NameError: name 'timestamps' is not defined